## **Demo Notebook**

This quick notebook will show how to

* load config
* create the dataset
* make a model
* load pretrained weights

This notebook should be run directly from the `scripts/` directory.

### **Config**

Make sure to change the following paths
* `DATASET_DIR`
* `LABELS_DIR`

In [1]:
from hydra import core, initialize, compose
from omegaconf import OmegaConf


# CHANGE ME
DATASET_DIR = 'D:/datasets/nuscenes'
LABELS_DIR = 'D:/datasets/cvt_labels_nuscenes'

core.global_hydra.GlobalHydra.instance().clear()        # required for Hydra in notebooks

initialize(config_path='../config')

# Add additional command line overrides
cfg = compose(
    config_name='config',
    overrides=[
        'experiment.save_dir=../logs/',                 # required for Hydra in notebooks
        '+experiment=cvt_nuscenes_vehicle',
        f'data.dataset_dir={DATASET_DIR}',
        f'data.labels_dir={LABELS_DIR}',
        'data.version=v1.0-trainval',
        'loader.batch_size=1',
    ]
)

# resolve config references
OmegaConf.resolve(cfg)

print(list(cfg.keys()))

C:\Users\79364\AppData\Local\Temp\ipykernel_20328\2568245292.py:11: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  initialize(config_path='../config')


['experiment', 'loader', 'optimizer', 'scheduler', 'trainer', 'model', 'data', 'visualization', 'loss', 'metrics']


### **Dataset Setup**

In [2]:
import torch
import numpy as np

from cross_view_transformer.common import setup_experiment, load_backbone


# Additional splits can be added to cross_view_transformer/data/splits/nuscenes/
SPLIT = 'mini_train'
#控制数据集采样率的参数 5个选1
SUBSAMPLE = 5


model, data, viz = setup_experiment(cfg)

dataset = data.get_split(SPLIT, loader=False)
dataset = torch.utils.data.ConcatDataset(dataset)
dataset = torch.utils.data.Subset(dataset, range(0, len(dataset), SUBSAMPLE))

loader = torch.utils.data.DataLoader(dataset, batch_size=1, shuffle=False, num_workers=2)

print(len(dataset))

Loaded pretrained weights for efficientnet-b4


C:\Users\79364\miniconda3\envs\transview\lib\site-packages\torch\utils\checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


65


### **Network Setup**

In [8]:
from pathlib import Path


# Download a pretrained model (13 Mb)
# MODEL_URL = 'https://www.cs.utexas.edu/~bzhou/cvt/cvt_nuscenes_vehicles_50k.ckpt'
CHECKPOINT_PATH = f'C:\\Users\\79364\PycharmProjects\\transview\logs\cross_view_transformers_test\model.ckpt'

# !wget $MODEL_URL -O $CHECKPOINT_PATH


if Path(CHECKPOINT_PATH).exists():
    network = load_backbone(CHECKPOINT_PATH)
else:
    network = model.backbone

    print(f'{CHECKPOINT_PATH} not found. Using randomly initialized weights.')

Loaded pretrained weights for efficientnet-b4


### **Run the Model**

This cell will save the model predictions and render them as a gif.

Note: colors will be ugly since gif only supports 256 colors total

In [9]:
import torch
import time
import imageio
import ipywidgets as widgets


GIF_PATH = './predictions.gif'


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

network.to(device)
network.eval()

images = list()

with torch.no_grad():
    for batch in loader:
        batch = {k: v.to(device) if isinstance(v, torch.Tensor) else v for k, v in batch.items()}
        pred = network(batch)

        visualization = np.vstack(viz(batch=batch, pred=pred))

        images.append(visualization)


# Save a gif
duration = [1 for _ in images[:-1]] + [5 for _ in images[-1:]]
imageio.mimsave(GIF_PATH, images, duration=duration)

html = f'''
<div align="center">
<img src="{GIF_PATH}?modified={time.time()}" width="80%">
</div>
'''

display(widgets.HTML(html))

HTML(value='\n<div align="center">\n<img src="./predictions.gif?modified=1740408474.6973128" width="80%">\n</d…